In [4]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append('../Scripts')



# used for weighted percentiles.  
from PostProcessingScripts import * 
import pandas as pd


# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))


%matplotlib inline
import pesummary
from pesummary.io import read
print(pesummary.__version__)
import h5py
# %config InlineBackend.figure_format = 'retina'



0.9.1


##   code to create csv files with credible intervals possible GWTC-2 BHNS detections
To make this original code work, you will have to download the hdf5 files from the GWTC-2 catalog that belongs to https://arxiv.org/abs/2010.14527
This script just creates csv files that summarize the credible intervals of interest for the possible BHNS binaries from GWTC-2. It is probably the most easiest to directly use those CSV files. 

The GWTC-2 data is available from https://dcc.ligo.org/LIGO-P2000223/public 

In [5]:




BHNSsGWTC2 = ['GW190425', 'GW190814', 'GW190426_152155'] 

def obtainGWTC1_MassAndWeight(dirGWTC='/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/', dfCSVname='/Users/floorbroekgaarden/Projects/BlackHole-NeutronStar/csvFiles/GWTC_Quantiles/'):
    
    print('calculating credible intervals for possible GWTC-2 BHNS systems')
    for ind_BHNS, BHNS_ in enumerate(BHNSsGWTC2):
        print(BHNS_)
        BHNS_file = dirGWTC + BHNS_ +'_comoving.h5'
        BHNS = h5.File(BHNS_file, 'r')

        data = read(BHNS_file)
        samples_dict = data.samples_dict
        posterior_samples = samples_dict['PublicationSamples']
        parameters = sorted(list(posterior_samples.keys()))

        mass_1_source = posterior_samples['mass_1_source']
        mass_2_source = posterior_samples['mass_2_source']
        total_mass_source = posterior_samples['total_mass_source']
        chirp_mass_source= posterior_samples['chirp_mass_source']
        # our mass ratio is defined the other way around
        symmetric_mass_ratio= 1./posterior_samples['symmetric_mass_ratio']



        # prepare DataFrame 
        xvarHeaders = ['mass_1_source', 'mass_2_source', \
                        'total_mass_source', 'chirp_mass_source', 'symmetric_mass_ratio']

        xvarUnits = ['Msun', 'Msun', 'Msun', 'Msun', '#']

        # quantiles that I want to know
        y_quantiles  =          [ 0.005, 0.05,   0.5,    0.95,  0.995]
        indexnames   = ['unit', '0.005', '0.05',  '0.5',  '0.95', '0.995']
        # nr of rows and columns that will be used:
        ncol_var = len(xvarHeaders)   
        ncol_Rate_det = 1

        nrows = len(y_quantiles) + 1 # +1 for units (see below)
        # store variables, and Observed and intrinsic rates for all MSSFR variations:
        ncol = ncol_var #* (ncol_MSSFR) # for each MSSFR I want to give quantiles for each xparam 
        df_placeholder = np.zeros((nrows, ncol)) # will be filled in loop: 

        headernames=[]
        units=[]
        for ind_s, ss in enumerate(xvarHeaders):
            sss = ss 
            headernames.append(sss)
            units.append(xvarUnits[ind_s])

        # store dataFrame with zeros that we will fill on the go:
        dfw = pd.DataFrame(data=df_placeholder, columns=headernames, index=indexnames)   
        # add units at first row (index=0)
        dfw.iloc[0]=units        



        # obtain BH and NS masses from combining GW posteriors.
        # by definition LIGO gives Most massive and least massive. 
        xvarlist = [mass_1_source, mass_2_source, total_mass_source, chirp_mass_source, symmetric_mass_ratio]


        # calculate quantiles with bootstrapping (if Nrepeats>1)
        for ind_xvar, xvar in enumerate(xvarlist):


                # calculate quantiles (credible intervals) from data 
                xvar_quantiles = weighted_quantile(values=xvar, quantiles=y_quantiles, \
                     sample_weight=np.ones_like(xvar))

                dfw_key = xvarHeaders[ind_xvar] 
                dfw[dfw_key][1:] = xvar_quantiles


        dfCSVname_ = dfCSVname + 'CredibleIntervals_' + BHNS_  + '.csv' 
        dfw.to_csv(dfCSVname_) 


        print()
    print('finished')



      
    return

In [6]:
obtainGWTC1_MassAndWeight(dirGWTC='/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/', dfCSVname='/Users/floorbroekgaarden/Projects/BlackHole-NeutronStar/csvFiles/GWTC_Quantiles/')

calculating credible intervals for possible GWTC-2 BHNS systems
GW190425

GW190814

GW190426_152155

finished
